In [19]:
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifier_trainer import ClassifierTrainer
from cs231n.gradient_check import eval_numerical_gradient
from cs231n.classifiers.convnet import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-18, np.abs(x) + np.abs(y))))


from cs231n.data_utils import load_CIFAR10
# Modify load_CIFAR10 and the following function to load less data if you have memory issues.
# Load batches 1, 2 and 3; and call the function as follows:
#def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
def get_CIFAR10_data(num_training=19000, num_validation=1000, num_test=1000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    
    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2).copy()
    X_val = X_val.transpose(0, 3, 1, 2).copy()
    x_test = X_test.transpose(0, 3, 1, 2).copy()

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: {}'.format(X_train.shape))
print('Train labels shape: {}'.format(y_train.shape))
print('Validation data shape: {}'.format(X_val.shape))
print('Validation labels shape: {}'.format(y_val.shape))
print('Test data shape: {}'.format(X_test.shape))
print('Test labels shape: {}'.format(y_test.shape))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape: (19000, 3, 32, 32)
Train labels shape: (19000,)
Validation data shape: (1000, 3, 32, 32)
Validation labels shape: (1000,)
Test data shape: (1000, 32, 32, 3)
Test labels shape: (1000,)


In [29]:
from cs231n.classifiers.convnet import *
model = init_affin_layer_convnet(filter_size=5, num_filters=23, weight_scale=3e-2)
trainer = ClassifierTrainer()
best_model, loss_history, train_acc_history, val_acc_history = trainer.train(
                  X_train[:50], y_train[:50], X_val, y_val, model, affin_layer_convnet,
                  reg=5.0, momentum=0.9, learning_rate=0.0001, batch_size=10, num_epochs=10,
                  acc_frequency=50, verbose=True)

starting iteration 0
Finished epoch 0 / 10: cost 130.266887, train: 0.120000, val 0.091000, lr 1.000000e-04
Finished epoch 1 / 10: cost 128.183494, train: 0.100000, val 0.111000, lr 9.500000e-05
Finished epoch 2 / 10: cost 124.058520, train: 0.160000, val 0.122000, lr 9.025000e-05
starting iteration 10
Finished epoch 3 / 10: cost 119.390604, train: 0.320000, val 0.109000, lr 8.573750e-05
Finished epoch 4 / 10: cost 115.640471, train: 0.360000, val 0.119000, lr 8.145062e-05
starting iteration 20
Finished epoch 5 / 10: cost 110.254990, train: 0.400000, val 0.125000, lr 7.737809e-05
Finished epoch 6 / 10: cost 106.060420, train: 0.380000, val 0.113000, lr 7.350919e-05
starting iteration 30
Finished epoch 7 / 10: cost 102.003324, train: 0.420000, val 0.109000, lr 6.983373e-05
Finished epoch 8 / 10: cost 98.111037, train: 0.480000, val 0.116000, lr 6.634204e-05
starting iteration 40
Finished epoch 9 / 10: cost 94.583301, train: 0.440000, val 0.133000, lr 6.302494e-05
Finished epoch 10 / 10: